In [51]:
#Importar paquetesgroup
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [52]:
#Importamos variables temporales
now = datetime.now()
ayer = datetime.now() - timedelta(days=1)
anteayer = datetime.now() - timedelta(days=2)

# la fecha de corte es la más actual
fecha_corte = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y") # es la fecha actual en formato dd/mm/yyyy
print(now.strftime("%d") + '-' + now.strftime("%m") + '-' + now.strftime("%Y"))

23-04-2024


In [53]:
# Para limpiar variables duplicadas luego de un merge
def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

# Para crear el COD_ID de la ejecutora
def crear_id(df, NIVEL_GOB, SECTOR, PLIEGO, EJECUTORA):
    df = df.assign(COD_ID = np.where(df[NIVEL_GOB].str.startswith('2'), (df[PLIEGO].str.split('. ', n=1).str[0] +"-" + df[EJECUTORA].str.split('. ', n=1).str[0]),
                                     (df[SECTOR].str.split('. ', n=1).str[0] +"-" + df[PLIEGO].str.split('. ', n=1).str[0] +"-"+ df[EJECUTORA].str.split('. ', n=1).str[0])))
    return df

In [54]:
bd_canon= pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Outputs', 'Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_CANON_123042024' + '.xlsx'), sheet_name='RD GASTO total_MOD')
bd_foncor=pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Outputs', 'Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_FONCOR_123042024' + '.xlsx'), sheet_name='RD GASTO total')
bd_rd_ant = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Outputs', 'Propuesta_RD_GG 2.3 PMG2025-2027_16042024_Mantenimiento_ingresos_170424' + '.xlsx'), sheet_name='RD GASTO total')

In [55]:
canon=bd_canon.copy()
canon = canon[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2023', 'PIM_2023', 'DEV_2023','PIA_2024', 'PIM_2024', 'DEV_2024', 'PMG_2025','PMG_2026','PMG_2027']]
canon ['TOTAL_GEN_PLIEGO25']= canon.groupby(['PLIEGO', 'GENERICA'])['PMG_2025'].transform('sum').astype(float)
canon ['TOTAL_GEN_PLIEGO26']= canon.groupby(['PLIEGO', 'GENERICA'])['PMG_2026'].transform('sum').astype(float)
canon ['TOTAL_GEN_PLIEGO27']= canon.groupby(['PLIEGO', 'GENERICA'])['PMG_2027'].transform('sum').astype(float)

canon ['TOTAL_PLIEGO25']= canon.groupby(['PLIEGO'])['PMG_2025'].transform('sum').astype(float)
canon ['TOTAL_PLIEGO26']= canon.groupby(['PLIEGO'])['PMG_2026'].transform('sum').astype(float)
canon ['TOTAL_PLIEGO27']= canon.groupby(['PLIEGO'])['PMG_2027'].transform('sum').astype(float)

canon ['IND_GEN_PLIEGO25']= (canon ['TOTAL_GEN_PLIEGO25'] / canon ['TOTAL_PLIEGO25'])*100
canon ['IND_GEN_PLIEGO26']= (canon ['TOTAL_GEN_PLIEGO26'] / canon ['TOTAL_PLIEGO26'])*100
canon ['IND_GEN_PLIEGO27']= (canon ['TOTAL_GEN_PLIEGO27'] / canon ['TOTAL_PLIEGO27'])*100

canon ['ALERTA_SUP20%_25'] = np.where(canon['GENERICA'].str.contains('BIENES Y SERVICIOS')&(canon['IND_GEN_PLIEGO25'] > 20), 'SI', 'NO')
canon ['ALERTA_SUP20%_26'] = np.where(canon['GENERICA'].str.contains('BIENES Y SERVICIOS')&(canon['IND_GEN_PLIEGO26'] > 20), 'SI', 'NO')
canon ['ALERTA_SUP20%_27'] = np.where(canon['GENERICA'].str.contains('BIENES Y SERVICIOS')&(canon['IND_GEN_PLIEGO27'] > 20), 'SI', 'NO')

canon=canon[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA', 'PIA_2023', 'PIM_2023', 'DEV_2023','PIA_2024', 'PIM_2024', 'DEV_2024', 'PMG_2025','PMG_2026','PMG_2027', 
                                     'TOTAL_GEN_PLIEGO25', 'TOTAL_PLIEGO25', 'IND_GEN_PLIEGO25', 'ALERTA_SUP20%_25', 'TOTAL_GEN_PLIEGO26', 'TOTAL_PLIEGO26', 'IND_GEN_PLIEGO26', 'ALERTA_SUP20%_26', 'TOTAL_GEN_PLIEGO27', 'TOTAL_PLIEGO27', 'IND_GEN_PLIEGO27', 'ALERTA_SUP20%_27']]
canon.info()
canon.head(139)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      141 non-null    object 
 1   SECTOR              141 non-null    object 
 2   PLIEGO              141 non-null    object 
 3   FUENTE              141 non-null    object 
 4   RUBRO               141 non-null    object 
 5   TIPO_GEN_ING        141 non-null    object 
 6   CATEGORIA_GTO       141 non-null    object 
 7   GENERICA            141 non-null    object 
 8   PIA_2023            86 non-null     float64
 9   PIM_2023            86 non-null     float64
 10  DEV_2023            86 non-null     float64
 11  PIA_2024            76 non-null     float64
 12  PIM_2024            76 non-null     float64
 13  DEV_2024            76 non-null     float64
 14  PMG_2025            141 non-null    int64  
 15  PMG_2026            141 non-null    int64  
 16  PMG_2027

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PIA_2023,PIM_2023,...,IND_GEN_PLIEGO25,ALERTA_SUP20%_25,TOTAL_GEN_PLIEGO26,TOTAL_PLIEGO26,IND_GEN_PLIEGO26,ALERTA_SUP20%_26,TOTAL_GEN_PLIEGO27,TOTAL_PLIEGO27,IND_GEN_PLIEGO27,ALERTA_SUP20%_27
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,10810654.0,22766248.0,...,99.997514,NO,11691535.0,11691820.0,99.997562,NO,11925271.0,11925556.0,99.997610,NO
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,25436520.0,70815105.0,...,7.092679,NO,21998440.0,312911400.0,7.030246,NO,21426080.0,306773231.0,6.984338,NO
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.0,335212830.0,...,20.383028,NO,62962440.0,312911400.0,20.121491,NO,61137396.0,306773231.0,19.929182,NO
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,674677812.0,327484750.0,...,72.524293,NO,227950520.0,312911400.0,72.848263,NO,224209755.0,306773231.0,73.086480,NO
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,6134251.0,9907527.0,...,3.497591,NO,2499086.0,68931195.0,3.625479,NO,1585750.0,66164452.0,2.396680,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,461. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TUMBES,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,NaN,NaN,...,78.885084,NO,28155060.0,36013388.0,78.179426,NO,26695095.0,34092655.0,78.301602,NO
135,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,NaN,NaN,...,3.135318,NO,5783042.0,162715808.0,3.554075,NO,4948158.0,156265536.0,3.166506,NO
136,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,NaN,NaN,...,90.076043,NO,146573894.0,162715808.0,90.079689,NO,140958506.0,156265536.0,90.204475,NO
137,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,NaN,NaN,...,7.400188,NO,2786706.0,61038927.0,4.565457,NO,3104166.0,59328086.0,5.232203,NO


In [56]:
bd_rd_act = pd.concat([canon, bd_foncor], ignore_index=True)
bd_rd_act.rename(columns={'PMG_2025': 'PMG_2025_ACT',
                         'PMG_2026': 'PMG_2026_ACT',
                         'PMG_2027': 'PMG_2027_ACT'}, inplace=True)
bd_rd_act.info()
bd_rd_act.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      215 non-null    object 
 1   SECTOR              215 non-null    object 
 2   PLIEGO              215 non-null    object 
 3   FUENTE              215 non-null    object 
 4   RUBRO               215 non-null    object 
 5   TIPO_GEN_ING        215 non-null    object 
 6   CATEGORIA_GTO       215 non-null    object 
 7   GENERICA            215 non-null    object 
 8   PIA_2023            137 non-null    float64
 9   PIM_2023            137 non-null    float64
 10  DEV_2023            137 non-null    float64
 11  PIA_2024            129 non-null    float64
 12  PIM_2024            129 non-null    float64
 13  DEV_2024            129 non-null    float64
 14  PMG_2025_ACT        215 non-null    int64  
 15  PMG_2026_ACT        215 non-null    int64  
 16  PMG_2027

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PIA_2023,PIM_2023,...,IND_GEN_PLIEGO25,ALERTA_SUP20%_25,TOTAL_GEN_PLIEGO26,TOTAL_PLIEGO26,IND_GEN_PLIEGO26,ALERTA_SUP20%_26,TOTAL_GEN_PLIEGO27,TOTAL_PLIEGO27,IND_GEN_PLIEGO27,ALERTA_SUP20%_27
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,10810654.0,22766248.0,...,99.997514,NO,11691535.0,11691820.0,99.997562,NO,11925271.0,11925556.0,99.997610,NO
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,25436520.0,70815105.0,...,7.092679,NO,21998440.0,312911400.0,7.030246,NO,21426080.0,306773231.0,6.984338,NO
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,4. DONACIONES Y TRANSFERENCIAS,0.0,335212830.0,...,20.383028,NO,62962440.0,312911400.0,20.121491,NO,61137396.0,306773231.0,19.929182,NO
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,674677812.0,327484750.0,...,72.524293,NO,227950520.0,312911400.0,72.848263,NO,224209755.0,306773231.0,73.086480,NO
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,6134251.0,9907527.0,...,3.497591,NO,2499086.0,68931195.0,3.625479,NO,1585750.0,66164452.0,2.396680,NO


In [57]:
bd_rd_ant.rename(columns={'PMG_2025': 'PMG_2025_ANT',
                         'PMG_2026': 'PMG_2026_ANT',
                         'PMG_2027': 'PMG_2027_ANT'}, inplace=True)
bd_rd_ant = bd_rd_ant[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE',	'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA', 'PMG_2025_ANT', 'PMG_2026_ANT', 'PMG_2027_ANT']]
bd_rd_ant.info()
bd_rd_ant.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  220 non-null    object
 1   SECTOR          220 non-null    object
 2   PLIEGO          220 non-null    object
 3   FUENTE          220 non-null    object
 4   RUBRO           220 non-null    object
 5   TIPO_GEN_ING    220 non-null    object
 6   CATEGORIA_GTO   220 non-null    object
 7   GENERICA        220 non-null    object
 8   PMG_2025_ANT    220 non-null    int64 
 9   PMG_2026_ANT    220 non-null    int64 
 10  PMG_2027_ANT    220 non-null    int64 
dtypes: int64(3), object(8)
memory usage: 19.0+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PMG_2025_ANT,PMG_2026_ANT,PMG_2027_ANT
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,836202,851782,884990
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,257351306,262146313,272366407
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0,0,0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,500,500,500
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,11462167,11691320,11925056


In [59]:
Hoja1 = bd_rd_ant.merge(bd_rd_act, on = ['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE',	'RUBRO', 'TIPO_GEN_ING', 'CATEGORIA_GTO', 'GENERICA'], how = "outer")
Hoja1.info()
Hoja1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      220 non-null    object 
 1   SECTOR              220 non-null    object 
 2   PLIEGO              220 non-null    object 
 3   FUENTE              220 non-null    object 
 4   RUBRO               220 non-null    object 
 5   TIPO_GEN_ING        220 non-null    object 
 6   CATEGORIA_GTO       220 non-null    object 
 7   GENERICA            220 non-null    object 
 8   PMG_2025_ANT        220 non-null    int64  
 9   PMG_2026_ANT        220 non-null    int64  
 10  PMG_2027_ANT        220 non-null    int64  
 11  PIA_2023            137 non-null    float64
 12  PIM_2023            137 non-null    float64
 13  DEV_2023            137 non-null    float64
 14  PIA_2024            129 non-null    float64
 15  PIM_2024            129 non-null    float64
 16  DEV_2024

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PMG_2025_ANT,PMG_2026_ANT,...,IND_GEN_PLIEGO25,ALERTA_SUP20%_25,TOTAL_GEN_PLIEGO26,TOTAL_PLIEGO26,IND_GEN_PLIEGO26,ALERTA_SUP20%_26,TOTAL_GEN_PLIEGO27,TOTAL_PLIEGO27,IND_GEN_PLIEGO27,ALERTA_SUP20%_27
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,836202,851782,...,0.323874,NO,774658.0,239184885.0,0.323874,NO,801658.0,247521649.0,0.323874,NO
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,257351306,262146313,...,99.676126,NO,238410227.0,239184885.0,99.676126,NO,246719991.0,247521649.0,99.676126,NO
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,0,0,...,0.002486,NO,285.0,11691820.0,0.002438,NO,285.0,11925556.0,0.002390,NO
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",1. SALDOS DE BALANCE,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,500,500,...,99.997514,NO,11691535.0,11691820.0,99.997562,NO,11925271.0,11925556.0,99.997610,NO
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,6. GASTOS DE CAPITAL,6. ADQUISICION DE ACTIVOS NO FINANCIEROS,11462167,11691320,...,99.997514,NO,11691535.0,11691820.0,99.997562,NO,11925271.0,11925556.0,99.997610,NO


In [60]:
Hoja2 = Hoja1.copy()
Hoja2 = Hoja2[Hoja2['GENERICA'] == "3. BIENES Y SERVICIOS" ]
outputFile = os.path.join(r'Y:\PMG 2025-2027\Outputs','Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_MOD_'+ fecha_corte + ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    Hoja1.to_excel(ew, sheet_name="RD GASTO total", index = False)
    Hoja2.to_excel(ew, sheet_name="RD GG2.3", index = False)
#%%
        